# Kinetic analysis of strategy enabling anaerobic production of PHB

In this script, we have performed the ECM analyses for the thermodynamic investigation of the NOG+EMP pathway, also demonstrating how the analysis of other pathways and metabolic engineering strategies could be analyzed by other researchers with the custom-made functions we have developed in this study.

In [1]:
from equilibrator_api import Q_ #for creating quantity variables
import equilibrator_custom_functions as eq #custom functions created during analysis
import numpy as np #numerical package
import math
import matplotlib.pyplot as plt #plotting package
import matplotlib.ticker as mtick

Fragments already downloaded
Fragments already downloaded


# Input Files and their formats

The script's input is based on the two custom __input csv files__, which are:

   __The compound file__, with metabolite names, IDs and bounds. It is currently name "metabolite_reference_table"
   
   This file uses a custom csv file format, which contains information of multiple metabolites, to create a dictionary which will be used for the eQuilibrator API for high throughput analysis. This dictionary allows for writing reactions in a simpler way (with acronyms), and customizing the ranges of metabolites according to data. The columns contain the following elements:

| Metabolite Acronym        | keggID                          | Lower Bound (M)             | Upper Bound (M)               |
|------------------|----------------------------------|----------------------|------------------------|
|Custom-defined name for metabolite| Kegg ID to identify metabolite (necessary to use specific kegg ID) | Custom-defined minimal concentration (in M)| Custom-defined maximal concentration (in M) |

Notice that the columns do not need to be mantained in order to be recognized in the script. More usefull information can be added to the other columns, as the script will ignore the information stored in these.

   __The pathway file__, with reaction names, formulae, fluxes, and, in the case of ECM, enzyme parameters (which will be read by other functions below). Being pathway specific, there are many files referenced in this script. The columns contain the follow basic setup:
   
| Reaction Formula        | Relative Flux                          | Reaction Name  |     Standard dG (kJ/mol)        | kcrf(1/s) | MWe(Da)        | kM(mM) |
|------------------|----------------------------------|----------------------|------------------------|-----------------|-------|-----|
|Using the acronyms defined in the compound file: S1 + S2 <=> P1 + P2| The relative stoichiometric coefficient of the reaction in the pathway | Customizable name to identify the reaction in graphs, for example | __(Optional)__ Standard Gibs Free Energy of Reaction in kJ/mol. These are custom values that can replace the values obtained from the eQuilibrator packages|__(Only needed for ECM)__ Forward catalytic rate of reaction's enzyme in 1/s (mind the direction of the reaction) |__(Only needed for ECM)__ Enzyme molecular weight, equivalent to Daltons |__(Only needed for ECM)__ Michaelis costants of each substrate and product (except water) in milimolar, with format example 'S1: 0.1 S2:0.02 P1:3 P2:0.3' | 

# Setting Intracellular Conditions

Below we set the intracellular conditions in *E. coli*, i.e the pH, ionic strengh, temperature and pMg (which have an effect on the thermodynamics of reactions). Note that metabolite concentration bounds are also organism specific, but are defined in the compound file.

In the cell bellow the name of the compound csv file is defined __("metabolite_reference_table")__. With the function __eq.obtain_compound_settings( string: compound_file_name )__, we construct a __CompoundSettings__ object containing data structures for idetifying compounds and setting their maximal and minimal concentrations for the model)

With __eq.change_bounds__ , one can create a new CompoundSettings object with specific changes in the metabolite concentration bounds.

The __eq.fix_concentrations__ function can also be used when the concentration needs to fixed at a specific concentration

In [2]:
#E. coli physiological conditions as a dictionary
E_coli_con = {
            "p_h": str(Q_("7.5")),
            "p_mg": str(Q_(10)),
            "ionic_strength": str(Q_('200 mM')),
            "temperature": str(Q_(37+273.15,"K")),
            "kcat_source": "fwd",
}

#Generating Compound Settings with the default eQuilibrator bounds
cs_default_bounds = eq.obtain_compound_settings("metabolite_reference_table", custom_bounds = False)
#Lowering the CoA lower concentration bound to 1 uM, 'freeing' CoA concentration for optimization at lower levels
cs_free_CoA = eq.change_bounds(cs_default_bounds, [('CoA',Q_(1e-6,'M'), Q_(1e-3,'M'))])
#Reducing AcCoA upper bound in order to represent conditions of lower size of CoA moiety of approximately 1 mM
cs_reduced_AcCoA_CoA = eq.change_bounds(cs_free_CoA, [('AcCoA',Q_(1e-6,'M'), Q_(1e-3,'M'))])
#NAD and NADH are released to bounds of 0.1 mM and 1 mM
cs_free_NAD = eq.change_bounds(cs_free_CoA, [('NAD',Q_(1e-4,'M'),Q_(1e-3,'M')),('NADH',Q_(1e-4,'M'),Q_(1e-3,'M'))])

# Calculating absolute flux of reactions

For ECM, an important step is the calculation of absolute fluxes of reactions, which will affect the enzyme concentrations/masses calculated. Note that with the same compound settings and kinetic parameters, __(enzyme cost)/flux is a constant__, meaning that changing absolute flux won't affect the relative levels of enzymes (though it would change total concentrations).

In this case the ATP consumption rate for cell maintenance (divided by the pathway's ATP yield of 2) is used to defined the __minimum pathway flux__, in a scenario in which cells are not growing, only surviving and accumulating PHB.

In [3]:
CDW__Cmol_ratio = Q_(24.56,'g/mol')

Volume_per_CDW = Q_(1.9, 'ul/mg').to('L/g')
Y_ATP = Q_(2,'mol/mol') #mol ATP/mol pathway net reaction
mATP_perCDW = Q_(3.2e-3,'mol/gram/hour')

flux_ATP = mATP_perCDW/Y_ATP/Volume_per_CDW
flux_ATP = flux_ATP.to('M/s')
print(flux_ATP)

0.00023391812865497077 molar / second


# Checking enzyme parameters according to Haldane Relationship

The kinetic parameters of the all enzymes involved in the reaction can be readily verified by __eq.check_parameters__. For all the enzymes whose parameters yield a Haldane Relationship outside of the % tolerance for K<sub>EQ</sub>, a message will be printed. Tolerance is defined to 0% by default.

In [4]:
(Keq,_) = eq.check_parameters("NOGEMP",cs_default_bounds,E_coli_con,flux_ATP, tolerance = 0.05)

# Enzyme Cost Minimization (ECM) analysis

The ECM analysis can be performed with the function __eq.ECM_analysis__, using as arguments the name of the pathway csv file, the CompoundSettings object, the physiological context dictionary and the caculated absolute flux of the pathway. The function returns a model object which will be an input in the __eq.myECM__ function, where an initial log-concentration vector needs to be provided. The MDF solution is a suitable set of concentrations to use. The number of iterations can also be determined. The function returns the log-concentration vector of the solution.

Here we perform ECM analysis for CoA moiety pools of 1 mM and 10 mM:

In [5]:
model_1mM = eq.ECM_analysis("NOGEMP",cs_reduced_AcCoA_CoA,E_coli_con,flux_ATP)
model_10mM = eq.ECM_analysis("NOGEMP",cs_free_CoA,E_coli_con,flux_ATP)

#Calculate MDF and obtain the log([metabolites]) of the MDF solution.
lnC_MDF_1mM = model_1mM.MDF()
lnC_MDF_10mM = model_10mM.MDF()

lnC_ECM_1mM = eq.myECM(model_1mM.ecf,'SLSQP',ln_conc0 = lnC_MDF_1mM,n_iter=1000)
lnC_ECM_10mM = eq.myECM(model_10mM.ecf,'SLSQP',ln_conc0 = lnC_MDF_10mM,n_iter=1000)

## Total enzyme mass concentration

With the resulting log-concentration vector, total enzyme mass concentration can be determined with the function __eq.total_enzyme_volume__ (with inputs of model object and log-concentration vector). The result in gram/liter was also normalized to cell dry weight by dividing it with the volume per CDW value.

In [6]:
print((eq.total_enzyme_volume(model_1mM,lnC_ECM_1mM)*Volume_per_CDW,eq.total_enzyme_volume(model_1mM,lnC_ECM_1mM)))
print((eq.total_enzyme_volume(model_10mM,lnC_ECM_10mM)*Volume_per_CDW,eq.total_enzyme_volume(model_10mM,lnC_ECM_10mM)))

(<Quantity(1.298457225210128, 'dimensionless')>, <Quantity(683.398539584278, 'gram / liter')>)
(<Quantity(0.28515433421929925, 'dimensionless')>, <Quantity(150.0812285364733, 'gram / liter')>)


## Molar concentrations of enzymes

Molar concentrations of each enzymes can be obtained in the followig manner:

In [7]:
enz_conc = dict(zip(model_1mM.reaction_ids,list(model_1mM.ecf.ECF(lnC_ECM_1mM).flat)))

print(enz_conc)

{'SucP': 3.44725519924841e-05, 'Pgm': 7.947299796810233e-06, 'Pgi': 4.925398809226925e-05, 'CscK': 3.7610743130579737e-06, 'Tal': 0.0002242823305418212, 'Tkt': 7.552587137556902e-05, 'RpiA': 1.3206521158055625e-05, 'AraD': 8.115452490927737e-06, 'Xpk': 0.00017651087070013995, 'TpiA': 9.059861750938204e-07, 'Fba': 2.5402808279720957e-05, 'Fbp': 7.868628755173816e-07, 'Fpk': 0.00043578514850527373, 'Pta': 0.0004630958472113984, 'Gap': 1.0953032311869553e-05, 'Pgk': 5.129902039361942e-06, 'Pgam': 1.691595062270157e-05, 'Eno': 1.699396145093512e-05, 'PykA': 2.347376303485192e-05, 'Pdh': 7.556045369992088e-06, 'PhaA': 0.001788580273609101, 'PhaB': 0.013545428443014035, 'PhaC': 0.000860490882701069}


## Mass concentrations of enzymes

With the __enzyme_cost_distribution__ function, we can obtain an ordered dictionary of individual enzyme mass concentrations (in grams/liter) and individual percentages of total enzyme mass concentration.

In [8]:
(totals_1mM,percents_1mM) = eq.enzyme_cost_distribution(model_1mM,lnC_ECM_1mM)
(totals_10mM,percents_10mM) = eq.enzyme_cost_distribution(model_10mM,lnC_ECM_10mM)

print(totals_1mM)
print(totals_10mM)

#Example of how to access specific value:
percents_1mM['PhaB']

{'PhaB': 365.726567961379, 'PhaA': 143.08642188872807, 'PhaC': 55.38979811946781, 'Fpk': 40.361548884261445, 'Pta': 36.81611985330617, 'Xpk': 16.34808382250556, 'Tal': 7.898999399352401, 'Tkt': 5.516636222885688, 'Pgi': 3.030597887317327, 'Pdh': 2.137454114263362, 'SucP': 1.9370126964576817, 'PykA': 1.173688151742596, 'Fba': 0.9944437357262363, 'Eno': 0.7757743402351882, 'Pgam': 0.483051885981866, 'Pgm': 0.463812363441642, 'Gap': 0.3891612380407252, 'RpiA': 0.2570781408627108, 'Pgk': 0.21094157185856308, 'AraD': 0.19926682046223967, 'CscK': 0.12422828456030487, 'TpiA': 0.04886889428456067, 'Fbp': 0.028983307156807234}
{'PhaB': 41.93491254367863, 'Fpk': 40.36155318013685, 'Xpk': 16.348146849614714, 'PhaA': 11.323023461475014, 'Pta': 8.699926235443717, 'Tal': 7.89895337980975, 'PhaC': 5.978967926039077, 'Tkt': 5.516622966346421, 'Pgi': 3.030523176619991, 'SucP': 1.9370448751746445, 'Pdh': 1.908308862312282, 'PykA': 1.1699129400442119, 'Fba': 0.9944676030868759, 'Eno': 0.7746986626839708,

0.5351585447985537

## Metabolite concentrations

Metabolite concentrations (in M) can be verified in the following manner:

In [9]:
conc_dict_1mM = dict(zip(model_1mM.compound_ids, np.exp(lnC_ECM_1mM).flatten()))
conc_dict_10mM = dict(zip(model_10mM.compound_ids, np.exp(lnC_ECM_10mM).flatten()))

print(conc_dict_1mM)
print(conc_dict_10mM)

#Example of how to access specific value:
conc_dict_1mM['NADH']

{'ATP': 0.005000000000000002, 'ADP': 0.0005000000000000001, 'NAD': 0.0010000000000000002, 'Pi': 0.010000000000000004, 'NADH': 0.00010000000000000009, 'CoA': 2.619226906251057e-05, 'CO2': 9.999999999999997e-06, 'AcCoA': 0.0010000000000000002, 'PYR': 0.00015074375430792808, 'PEP': 0.000404324071758153, 'GADP': 0.0019688114495910614, 'DAHP': 0.0008509987194674273, 'R5P': 0.00022386467109881446, '3PG': 0.003612570965315041, 'AcAcCoA': 1.1103799145202133e-06, 'Ru5P': 4.816354446971689e-05, 'G6P': 0.010000000000000004, 'Sucrose': 0.004278855640792367, 'X5P': 4.91368634391264e-05, 'E4P': 0.010000000000000004, 'BPG': 5.4104072348835265e-05, 'S7P': 0.0013079292560448806, '2PG': 0.0003136685501593223, 'AcP': 0.010000000000000004, 'F16BP': 5.495420973875961e-05, 'HBCoA': 4.3238313851751535e-06, 'Fructose': 0.0001759510361367728, 'PHB': 1.000000298433069e-06, 'G1P': 0.0015298471784791462, 'F6P': 0.0015962234859388917}
{'ATP': 0.005000000000000002, 'ADP': 0.0005000000000000001, 'NAD': 0.00100000000

0.00010000000000000009

## Plotting enzyme demand distribution:

We use the function __eq.ECM_plot__ for plotting these graphs. The main difference from the original eQuilibrator plotting function is that in our case the measured concentrations of metabolites are not mandatory for plotting the graph.

In [10]:
fig1 = plt.figure(figsize=(15, 5))

ECM_1mM_mass_conc= round(eq.total_enzyme_volume(model,lnC_ECM_1mM).m_as('g/L'),2)
ax_1mM = fig1.add_subplot(1, 2, 1)
ax_1mM.set_title("[AcCoA] + [CoA] = 1 mM solution: " + str(ECM_1mM_mass_conc) + ' g/L')
ax_1mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_1mM,lnC_ECM_1mM,ax_1mM)
ax_1mM.annotate("A", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

ECM_10mM_mass_conc= round(eq.total_enzyme_volume(model,lnC_ECM_10mM).m_as('g/L'),2)
ax_10mM = fig1.add_subplot(1, 2, 2, sharey=ax_1mM)
ax_10mM.set_title("[AcCoA] + [CoA] = 10 mM solution: " + str(ECM_10mM_mass_conc) + ' g/L')
ax_10mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_10mM,lnC_ECM_10mM,ax_10mM)
ax_10mM.annotate("B", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

NameError: name 'model' is not defined

<Figure size 1080x360 with 0 Axes>

## Enzyme cost partition values

The partions of enzyme cost can be obtained by the __model.ecf.GetEnzymeCostPartitions__ function, and returns a 2D array with basic capacity concentrations (blue bars) on 1st column, fold increase in demand due to thermodynamics (orange bar) in the 2nd column and fold increase in demand due to saturation (brown bar) in 3rd. The 4th column is related to regulation fold change, which was not analyzed in our study.

In [ ]:
print(model_1mM.ecf.GetEnzymeCostPartitions(lnC_ECM_1mM))

## Optmizing NADH/NAD

We also analyze the ECM when the NADH/NAD ratio is free to be optimized:

In [ ]:
model_free_NAD = eq.ECM_analysis("NOGEMP",cs_free_NAD,E_coli_con,flux_ATP)

#Previous solutions (of [AcCoA] + [CoA] ~ 1 mM)
lnC_ECM_free_NAD = eq.myECM(model_free_NAD.ecf,'SLSQP',ln_conc0 = lnC_ECM_1mM,n_iter=1000)

print((eq.total_enzyme_volume(model_free_NAD,lnC_ECM_free_NAD)*Volume_per_CDW,eq.total_enzyme_volume(model_free_NAD,lnC_ECM_free_NAD)))
conc_dict_free_NAD = dict(zip(model_1mM.compound_ids, np.exp(lnC_ECM_free_NAD).flatten()))
#
conc_dict_free_NAD['CoA']

In [ ]:
fig1 = plt.figure(figsize=(15, 5))

ECM_10mM_mass_conc= round(eq.total_enzyme_volume(model_10mM,lnC_ECM_10mM).m_as('g/L'),2)
ax_10mM = fig1.add_subplot(1, 2, 1)
ax_10mM.set_title("[AcCoA] + [CoA] = 10 mM solution: " + str(ECM_10mM_mass_conc) + ' g/L')
ax_10mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_10mM,lnC_ECM_10mM,ax_10mM)
ax_10mM.annotate("A", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

ECM_NAD_mass_conc= round(eq.total_enzyme_volume(model_free_NAD,lnC_ECM_free_NAD).m_as('g/L'),2)
ax_NAD = fig1.add_subplot(1, 2, 2,sharey=ax_10mM)
ax_NAD.set_title("Free NAD solution: " + str(ECM_NAD_mass_conc) + ' g/L')
ax_NAD.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_free_NAD,lnC_ECM_free_NAD,ax_NAD)
ax_NAD.annotate("B", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')


# Parameter sweep of K<sub>M</sub> values of acetoacetyl-CoA reductase

Due to a significant portion of the enzyme cost being of acetoacetyl-CoA reductase (PhaB), we have performed a K<sub>M</sub> parameter sweep for the substrates and products of the enzyme, with the function __eq.ECM_kM_sweep__:

In [ ]:
#Range representing fold change of Km value
km_values = np.logspace(-3,1, 60)
tevs_AcAcCoA, lnCs_AcAcCoA = eq.ECM_kM_sweep(('AcAcCoA', 'PhaB',km_values),"NOGEMP",cs_free_CoA,E_coli_con,flux_ATP,n_iter = 500)
tevs_NADH, lnCs_NADH = eq.ECM_kM_sweep(('NADH', 'PhaB',km_values),"NOGEMP",cs_free_CoA,E_coli_con,flux_ATP,n_iter = 500)
tevs_HBCoA, lnCs_HBCoA = eq.ECM_kM_sweep(('HBCoA', 'PhaB',km_values),"NOGEMP",cs_free_CoA,E_coli_con,flux_ATP,n_iter = 500)
tevs_NAD, lnCs_NAD = eq.ECM_kM_sweep(('NAD', 'PhaB',km_values),"NOGEMP",cs_free_CoA,E_coli_con,flux_ATP,n_iter = 500)

In [ ]:
sp = plt.subplot()
plt.plot(km_values, tevs_AcAcCoA*Volume_per_CDW.magnitude*100, label = '$K_{M AcAcCOA}$')
plt.plot(km_values, tevs_NADH*Volume_per_CDW.magnitude*100, label = '$K_{M NADH}$')
plt.plot(km_values, tevs_HBCoA*Volume_per_CDW.magnitude*100, label = '$K_{M HBCoA}$')
plt.plot(km_values, tevs_NAD*Volume_per_CDW.magnitude*100, label = '$K_{M NAD}$')
sp.set_ylim(0,100)
sp.set_title('$K_{M}$ parameter sweep in NOG+EMP [AcCoA] + [CoA] = 10 mM', fontsize=11)
plt.xlabel('Fold change in $K_{M}$', fontsize = 12)
plt.ylabel('Minimized total enzyme cost (% of CDW)', fontsize = 11)
plt.legend()
plt.xscale("log")

# Substrate Channeling in ECM

We perform the same ECM analysis with implemented substrate channeling of ketothiolase and acetoacetyl-CoA reductase:

In [ ]:
model_1mM_SC = eq.ECM_analysis("NOGEMP_SC",cs_reduced_AcCoA_CoA,E_coli_con,flux_ATP)
model_10mM_SC = eq.ECM_analysis("NOGEMP_SC",cs_free_CoA,E_coli_con,flux_ATP)
model_free_NAD_SC = eq.ECM_analysis("NOGEMP_SC",cs_free_NAD,E_coli_con,flux_ATP)

#Calculate MDF and obtain the log([metabolites]) of the MDF solution.
lnC_MDF_1mM = model_1mM_SC.MDF()
lnC_MDF_10mM = model_10mM_SC.MDF()

lnC_ECM_1mM_SC = eq.myECM(model_1mM_SC.ecf,'SLSQP',ln_conc0 = lnC_MDF_1mM,n_iter=1000)
lnC_ECM_10mM_SC = eq.myECM(model_10mM_SC.ecf,'SLSQP',ln_conc0 = lnC_MDF_10mM,n_iter=1000)
lnc_ECM_free_NAD_SC = eq.myECM(model_free_NAD_SC.ecf, 'SLSQP',ln_conc0 = lnC_MDF_10mM,n_iter=1000)

In [ ]:
fig1 = plt.figure(figsize=(15, 5))

ECM_1mM_SC_mass_conc= round(eq.total_enzyme_volume(model_1mM_SC,lnC_ECM_1mM_SC).m_as('g/L'),2)
ax_1mM = fig1.add_subplot(1, 2, 1)
ax_1mM.set_title("[AcCoA] + [CoA] = 1 mM solution: " + str(ECM_1mM_SC_mass_conc) + ' g/L')
ax_1mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_1mM_SC,lnC_ECM_1mM_SC,ax_1mM)
ax_1mM.annotate("A", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

ECM_10mM_SC_mass_conc= round(eq.total_enzyme_volume(model_10mM_SC,lnC_ECM_10mM_SC).m_as('g/L'),2)
ax_10mM = fig1.add_subplot(1, 2, 2, sharey=ax_1mM)
ax_10mM.set_title("[AcCoA] + [CoA] = 10 mM solution: " + str(ECM_10mM_SC_mass_conc) + ' g/L')
ax_10mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_10mM_SC,lnC_ECM_10mM_SC,ax_10mM)
ax_10mM.annotate("B", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

We do the same to the case in which substrate channeling is implemented between the three enzymes from *phaCAB*:

In [ ]:
model_1mM_SC2 = eq.ECM_analysis("NOGEMP_SC2",cs_reduced_AcCoA_CoA,E_coli_con,flux_ATP)
model_10mM_SC2 = eq.ECM_analysis("NOGEMP_SC2",cs_free_CoA,E_coli_con,flux_ATP)
model_free_NAD_SC2 = eq.ECM_analysis("NOGEMP_SC2",cs_free_NAD,E_coli_con,flux_ATP)

#Calculate MDF and obtain the log([metabolites]) of the MDF solution.
lnC_MDF_1mM = model_1mM_SC2.MDF()
lnC_MDF_10mM = model_10mM_SC2.MDF()

lnC_ECM_1mM_SC2 = eq.myECM(model_1mM_SC2.ecf,'SLSQP',ln_conc0 = lnC_MDF_1mM,n_iter=1000)
lnC_ECM_10mM_SC2 = eq.myECM(model_10mM_SC2.ecf,'SLSQP',ln_conc0 = lnC_MDF_10mM,n_iter=1000)
lnc_ECM_free_NAD_SC2 = eq.myECM(model_free_NAD_SC2.ecf, 'SLSQP',ln_conc0 = lnC_MDF_10mM,n_iter=1000)

In [ ]:
fig1 = plt.figure(figsize=(15, 6))

ECM_1mM_SC2_mass_conc= round(eq.total_enzyme_volume(model_1mM_SC2,lnC_ECM_1mM_SC2).m_as('g/L'),2)
ax_1mM = fig1.add_subplot(1, 2, 1)
ax_1mM.set_title("[AcCoA] + [CoA] = 1 mM solution: " + str(ECM_1mM_SC2_mass_conc) + ' g/L')
ax_1mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_1mM_SC2,lnC_ECM_1mM_SC2,ax_1mM)
ax_1mM.annotate("A", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')

ECM_10mM_SC2_mass_conc= round(eq.total_enzyme_volume(model_10mM_SC2,lnC_ECM_10mM_SC2).m_as('g/L'),2)
ax_10mM = fig1.add_subplot(1, 2, 2, sharey=ax_1mM)
ax_10mM.set_title("[AcCoA] + [CoA] = 10 mM solution: " + str(ECM_10mM_SC2_mass_conc) + ' g/L')
ax_10mM.legend(loc="center left", bbox_to_anchor=(1, 0.5));
eq.ECM_plot(model_10mM_SC2,lnC_ECM_10mM_SC2,ax_10mM)
ax_10mM.annotate("B", xy=(0, 1.02), xycoords="axes fraction",fontsize = 18, fontweight = 'demibold')
plt.tight_layout()

plt.savefig('ECM_SC2.png', dpi =300)